In [2]:
import pandas as pd
import numpy as np

In [3]:
path_to_file = "/Users/aditgupta/Google Drive/Research/T2D/Summer_2018/drug_names.tsv" #this is the path to the file for all the drug names and corresponding ID's
df5 = pd.read_csv(path_to_file,delimiter="\t", header=None) #reads file, converts to csv, and explains to fucntio that the delimeter are tabs
df5.rename(columns={0:'ID'}, inplace=True) #renames index zero to 'ID'
df5.rename(columns={1:'drug_name'}, inplace=True) #renames index one to drug name
#OVERALL: df5 contains the drug name based on the 'CID' ID 

In [4]:
import xml.etree.ElementTree as ET
import lxml.html
#importing libraries that parse the xml file with detailed drug information 

In [5]:
path_to_file = '/Users/aditgupta/Google Drive/Research/T2D/Summer_2018/drugbank_full_database.xml' #file location of the drugbank database

In [6]:
tree = ET.parse(path_to_file) # function that parses the xml file from drugbank

In [7]:
def find_categories_matching_string(drug_element, category_name):
    all_categories = drug_element.findall(".//*{http://www.drugbank.ca}category")
    categories_wanted = list(filter(lambda category: category.text == category_name, all_categories))
    return categories_wanted

In [8]:
all_drugs = tree.findall("{http://www.drugbank.ca}drug")

In [9]:
def get_id_and_name_from_target(target):
    id = target.find("{http://www.drugbank.ca}id").text
    name = target.find("{http://www.drugbank.ca}name").text
    effect = target.find("*{http://www.drugbank.ca}category") #Blood Glucose Lowering
    gene = target.find("*{http://www.drugbank.ca}gene-name")
    #glucose_categories = find_categories_matching_string(target, "Blood Glucose Lowering Agents")
    #diabetes_categories = find_categories_matching_string(target, "Drugs Used in Diabetes")

    
    if gene is None:
        gene_found = None
        
    else:
        gene_found= gene.text
    return id, name, gene_found
#function that returns the target id, target name, and gene name of a particular sample

In [10]:
new_results = []
total_found = 0
for drug in all_drugs:
    drug_name = drug.find("{http://www.drugbank.ca}name").text
    drug_id = drug.find("{http://www.drugbank.ca}drugbank-id[@primary]").text
    element_containing_all_targets = drug.find("{http://www.drugbank.ca}targets")
    list_of_targets = element_containing_all_targets.findall("{http://www.drugbank.ca}target")
    cholesterol_categories = find_categories_matching_string(drug, "Anticholesteremic Agents")
    cholesterol_categories2 = find_categories_matching_string(drug, "Hypolipidemic Agents")
    cholesterol_categories3 = find_categories_matching_string(drug, "Lipid Modifying Agents")
    cholesterol_categories4 = find_categories_matching_string(drug, "Lipid Regulating Agents")
    if cholesterol_categories == []:
        cholesterol_categories = None
    if cholesterol_categories2 == []:
        cholesterol_categories2 = None
    if cholesterol_categories3 == []:
        cholesterol_categories3 = None
    if cholesterol_categories4 == []:
        cholesterol_categories4 = None
    for target in list_of_targets:
        target_id, target_name, target_gene = get_id_and_name_from_target(target)
        new_results.append([drug_name, drug_id, target_id, target_name, target_gene, cholesterol_categories,cholesterol_categories2,cholesterol_categories3,cholesterol_categories4])
    total_found += 1
#creates an array and list that contains the drug name, drug id, target id, target name, and target gene as extracted from drug bank

In [11]:
drug = all_drugs[0]
element_containing_all_targets = drug.find("{http://www.drugbank.ca}targets")
list_of_targets = element_containing_all_targets.findall("{http://www.drugbank.ca}target")
target = list_of_targets[0]

In [12]:
second_results = []
total_found = 0
for drug in all_drugs:
    drug_name = drug.find("{http://www.drugbank.ca}name").text
    drug_id = drug.find("{http://www.drugbank.ca}drugbank-id[@primary]").text
    element_containing_all_targets = drug.find("{http://www.drugbank.ca}targets")
    list_of_targets = element_containing_all_targets.findall("{http://www.drugbank.ca}target")
    for target in list_of_targets:
        target_id, target_name, target_gene = get_id_and_name_from_target(target)
        second_results.append([drug_name, drug_id, target_id, target_name, target_gene])
    total_found += 1
#creates an array and list that contains the drug name, drug id, target id, target name, and target gene as extracted from drug bank

In [13]:
df = pd.DataFrame(new_results) #df contains the results from the parsing of the xml file from drugbank
df.columns = ["drug_name", "drug_id", "target_id", "target_name", "target_gene","target_effect_cholesterol",'hypolipidemic','lipid_modifying','lipid_regulate'] #renaming the columns of the dataframe

In [14]:
df_category1 = df.dropna(subset = ['target_effect_cholesterol'])
df_category1.head()

,drug_name,drug_id,target_id,target_name,target_gene,target_effect_cholesterol,hypolipidemic,lipid_modifying,lipid_regulate
1287,Pravastatin,DB00175,BE0000574,3-hydroxy-3-methylglutaryl-coenzyme A reductase,HMGCR,[[]],[[]],[[]],[[]]
1457,Lovastatin,DB00227,BE0000574,3-hydroxy-3-methylglutaryl-coenzyme A reductase,HMGCR,[[]],[[]],[[]],[[]]
1458,Lovastatin,DB00227,BE0000516,Integrin alpha-L,ITGAL,[[]],[[]],[[]],[[]]
1459,Lovastatin,DB00227,BE0002366,Histone deacetylase 2,HDAC2,[[]],[[]],[[]],[[]]
2401,Cerivastatin,DB00439,BE0000574,3-hydroxy-3-methylglutaryl-coenzyme A reductase,HMGCR,[[]],[[]],[[]],[[]]


In [15]:
df_category2 = df.dropna(subset = ['hypolipidemic'])
df_category3 = df.dropna(subset = ['lipid_modifying'])
df_category4 = df.dropna(subset = ['lipid_regulate'])


In [16]:
df3 = df['drug_name'].value_counts() 
df4 = df3.to_frame(name=None)
df5 = df4.reset_index()
df6 = df5.rename(columns={'drug_name':'number_of_targets'}, inplace=False)
df7 = df6.rename(columns={'index':'drug_name'}, inplace=False)
df7.head()

,drug_name,number_of_targets
0,Fostamatinib,300
1,Artenimol,192
2,Copper,147
3,NADH,144
4,Zinc,124


In [17]:
target_freq = df['target_id'].value_counts() #number of drugs that target a given target ID in a series
df8 = target_freq.to_frame(name=None)
df9 = df8.reset_index()
df10 = df9.rename(columns={'target_id':'target_freq'}, inplace=False)
df11 = df10.rename(columns={'index':'target_id'}, inplace=False)
df11.head(10)

,target_id,target_freq
0,BE0001072,137
1,BE0000048,112
2,BE0000123,111
3,BE0000756,108
4,BE0000451,105
5,BE0000442,104
6,BE0004796,101
7,BE0000322,100
8,BE0001739,99
9,BE0000092,97


In [18]:
df12 = df7[(df7['number_of_targets'])<3]  
df12.head()

,drug_name,number_of_targets
1508,Equol,2
1509,Dimethyl fumarate,2
1510,"3,7,3',4'-TETRAHYDROXYFLAVONE",2
1511,Ardeparin,2
1512,Cyanamide,2


In [19]:
df13 = df11[(df11['target_freq'])>1]
df13.head()

,target_id,target_freq
0,BE0001072,137
1,BE0000048,112
2,BE0000123,111
3,BE0000756,108
4,BE0000451,105


In [20]:
df14 = pd.merge(df_category1, df12, on='drug_name', how='inner')
df15 = df14.dropna(subset= ['target_gene'])
df15

,drug_name,drug_id,target_id,target_name,target_gene,target_effect_cholesterol,hypolipidemic,lipid_modifying,lipid_regulate,number_of_targets
0,Pravastatin,DB00175,BE0000574,3-hydroxy-3-methylglutaryl-coenzyme A reductase,HMGCR,[[]],[[]],[[]],[[]],1
1,Cerivastatin,DB00439,BE0000574,3-hydroxy-3-methylglutaryl-coenzyme A reductase,HMGCR,[[]],[[]],[[]],[[]],1
2,Clofibrate,DB00636,BE0000071,Peroxisome proliferator-activated receptor alpha,PPARA,[[]],[[]],[[]],[[]],1
4,Fluvastatin,DB01095,BE0000574,3-hydroxy-3-methylglutaryl-coenzyme A reductase,HMGCR,[[]],[[]],[[]],[[]],1
5,Rosuvastatin,DB01098,BE0000574,3-hydroxy-3-methylglutaryl-coenzyme A reductase,HMGCR,[[]],[[]],[[]],[[]],1
7,Probucol,DB01599,BE0000119,ATP-binding cassette sub-family A member 1,ABCA1,[[]],[[]],[[]],[[]],2
8,Probucol,DB01599,BE0002705,Liver carboxylesterase 1,CES1,[[]],[[]],[[]],[[]],2
10,Torcetrapib,DB06281,BE0003516,Cholesteryl ester transfer protein,CETP,[[]],[[]],None,[[]],1
11,Mevastatin,DB06693,BE0000574,3-hydroxy-3-methylglutaryl-coenzyme A reductase,HMGCR,[[]],[[]],None,[[]],1
12,Pitavastatin,DB08860,BE0000574,3-hydroxy-3-methylglutaryl-coenzyme A reductase,HMGCR,[[]],[[]],[[]],[[]],1


In [21]:
df16 = pd.merge(df15, df13, on='target_id', how='outer')  
df17 = df16.dropna(subset = ['target_gene'])
df17

,drug_name,drug_id,target_id,target_name,target_gene,target_effect_cholesterol,hypolipidemic,lipid_modifying,lipid_regulate,number_of_targets,target_freq
0,Pravastatin,DB00175,BE0000574,3-hydroxy-3-methylglutaryl-coenzyme A reductase,HMGCR,[[]],[[]],[[]],[[]],1.0,16.0
1,Cerivastatin,DB00439,BE0000574,3-hydroxy-3-methylglutaryl-coenzyme A reductase,HMGCR,[[]],[[]],[[]],[[]],1.0,16.0
2,Fluvastatin,DB01095,BE0000574,3-hydroxy-3-methylglutaryl-coenzyme A reductase,HMGCR,[[]],[[]],[[]],[[]],1.0,16.0
3,Rosuvastatin,DB01098,BE0000574,3-hydroxy-3-methylglutaryl-coenzyme A reductase,HMGCR,[[]],[[]],[[]],[[]],1.0,16.0
4,Mevastatin,DB06693,BE0000574,3-hydroxy-3-methylglutaryl-coenzyme A reductase,HMGCR,[[]],[[]],None,[[]],1.0,16.0
5,Pitavastatin,DB08860,BE0000574,3-hydroxy-3-methylglutaryl-coenzyme A reductase,HMGCR,[[]],[[]],[[]],[[]],1.0,16.0
6,Clofibrate,DB00636,BE0000071,Peroxisome proliferator-activated receptor alpha,PPARA,[[]],[[]],[[]],[[]],1.0,45.0
7,Probucol,DB01599,BE0000119,ATP-binding cassette sub-family A member 1,ABCA1,[[]],[[]],[[]],[[]],2.0,3.0
8,Probucol,DB01599,BE0002705,Liver carboxylesterase 1,CES1,[[]],[[]],[[]],[[]],2.0,19.0
9,Torcetrapib,DB06281,BE0003516,Cholesteryl ester transfer protein,CETP,[[]],[[]],None,[[]],1.0,NaN


In [22]:
df17['target_gene'].unique()

array(['HMGCR', 'PPARA', 'ABCA1', 'CES1', 'CETP'], dtype=object)

In [23]:
df18 = pd.merge(df_category2, df12, on='drug_name', how='inner')
df19 = df18.dropna(subset= ['target_gene'])
df20 = pd.merge(df19, df13, on='target_id', how='outer')  
df21 = df20.dropna(subset = ['target_gene'])
df21['target_gene'].unique()

array(['HMGCR', 'inhA', 'katG', 'PPARA', 'ABCA1', 'CES1', 'SERPIND1',
       'SERPINC1', 'CETP', 'BCL2'], dtype=object)

In [24]:
df22 = pd.merge(df_category3, df12, on='drug_name', how='inner')
df23 = df22.dropna(subset= ['target_gene'])
df24 = pd.merge(df23, df13, on='target_id', how='outer')  
df25 = df24.dropna(subset = ['target_gene'])
df25['target_gene'].unique()

array(['HMGCR', 'AGTR1', 'ACE', 'BDKRB1', 'PPARA', 'SFRP4', 'ABCA1',
       'CES1', 'MTTP'], dtype=object)

In [25]:
df26 = pd.merge(df_category4, df12, on='drug_name', how='inner')
df27 = df26.dropna(subset= ['target_gene'])
df28 = pd.merge(df27, df13, on='target_id', how='outer')  
df29 = df28.dropna(subset = ['target_gene'])
df29['target_gene'].unique()

array(['HMGCR', 'inhA', 'katG', 'PPARA', 'ABCA1', 'CES1', 'SERPIND1',
       'SERPINC1', 'CETP', 'BCL2'], dtype=object)